<a href="https://colab.research.google.com/github/parkrye/Python/blob/main/2022_MLStudy/Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Final Exam Lap Session

**RULES**
* **TODO** 1~4 번에 해당하는 모델링 부분만 작성하고, 나머지 코드는 일체 수정하지 않는 것을 원칙으로 함.
  * 수정 시 감점 요소가 될 수 있으며, 부득이하게 수정 시 수정 사항을 주석으로 필히 명시해야함. 
  * `utils.py`, `train.py`은 절대 수정 불가.
* 완성된 `.ipynb`을 다운 받아서 이루리에 제출해야함.
* 인터넷을 참조하는 것은 얼마든지 가능하나 수강생 간 코드 공유는 학칙에 따라 `F` 부여.

---

* In principle, only the modeling parts corresponding to **TODO** 1 to 4 can be modified, and the rest of the code should not be modified.
   * Points may be deducted if modify other codes, and modification must be clearly stated in comments when unavoidable corrections are made.
   * Never modified `utils.py` and `train.py`.
* You must download the completed `.ipynb` and submit it to `E-RURI`.
* It is possible to refer any resources in Internet, but the code sharing among students will be given an ‘F’ according to the school regulations.


# Preparation

In [ ]:
!git clone https://github.com/choi-hyunsoo/machine_learnig_final.git

fatal: destination path 'machine_learnig_final' already exists and is not an empty directory.


In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import _LRScheduler
import torch.utils.data as data

import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

from sklearn import decomposition
from sklearn import manifold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from tqdm.notebook import tqdm, trange
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
from machine_learnig_final.utils import *
from machine_learnig_final.train import *

In [ ]:
# set random seed
set_seed()

# load data
train_iterator, valid_iterator, test_iterator = load_MNIST_data()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

# TODO
본 시험은 `MNIST` 데이터를 이용한 classifcation 문제를 다양한 Neural Networks 모델들로 구현하고 그 결과를 비교하는 것을 목표로 한다. 각 모델에 대한 그림과 설명을 참조하여 각 모델의 class를 완성하시오.

---

This exam aims to implement the classifcation problem using `MNIST` data with various Neural Networks models and compare the results. Refer to the picture and description for each model and complete the class of each model.

### 1. Multi-layer Perceptron [pts. 20]
아래와 같은 구조의 MLP를 구축하시오.
각 layer 사이에는 `ReLu` activation을 사용하시오.

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/mlp.png?raw=true" align="center" width="50%"/>
</div>

In [ ]:
class MLP(nn.Module):
    def __init__(self, output_dim, input_dim=32*32*1):
        super().__init__()
        self.output_dim = output_dim

        #### complete the code ####

        self.fc1 = nn.Linear(1024, 1024, bias=True)
        self.fc2 = nn.Linear(1024, 512, bias=True)
        self.fc3 = nn.Linear(512, 1024, bias=True)
    
        ###########################

    def forward(self, x):   
        #### complete the code ####

        out = F.relu(self.fc1(x)) 
        out = F.relu(self.fc2(out))  
        out = F.relu(self.fc3(out))  

        ###########################
        return out

### 2. Convolutional Neural Networks [pts. 20]
아래 그림과 같은 CNN 모델을 완성하시오.
Fully connectec Layer 직전에 `nn.Dropout(0.5)`를 추가하시오.

(힌트, `Lab5-2`를 참조하시오.)

---

Bulild the CNN model as shown in the figure below.
Add `nn.Dropout(0.5)` just before the Fully connectec Layer.

(Hint, refer to `Lab5-2`.)

---

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/cnn.png?raw=true" align="center" width="80%"/>
</div>

In [ ]:
class CNN(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.output_dim = output_dim

        #### complete the code ####

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(384*2*2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
        )

        ###########################

    def forward(self, x):
        #### complete the code ####

        out = self.features(x)
        h = out.view(out.shape[0], -1)
        out = self.classifier(h)

        ###########################
        return out

### 3. Convolutional Neural Networks with Skip Connection [pts. 20]
Deep architectures에서 skip connection은 특정 layer의 output을 몇 개의 layer를 건너뛰고 다음 layer의 input에 추가하는 것이다.

본 모델에서는 input data를 flatten하여 1d vector로 만든 뒤, CNN의 feature와 concate ([`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html))하여 최종적으로 classification을 진행한다.

**앞선 2번 모델에 Skip Connection만 추가하는 것을 목표로 한다.**

---

In deep architectures, skip connection is where the output of a specific layer is skipped several layers and added to the input of the next layer.

In this model, after flattening the input data to make a 1d vector, concatenating it with the CNN feature ([`torch.cat`](https://pytorch.org/docs/stable/generated/torch.cat.html)). 

**Add only Skip Connection to the previous model No. 2**

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/cnn_SC.png?raw=true" align="center" width="80%"/>
</div>

In [ ]:
class CNNwithSC(nn.Module):
    def __init__(self, output_dim, input_dim = 32 * 32 * 1):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim

        #### complete the code ####

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(2),
            nn.ReLU(inplace=True),
        )

        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(384*2*2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, output_dim),
        )

        ###########################

    def forward(self, x):
        #### complete the code ####

        f = torch.flatten(x, 1)
        out = self.features(x)
        torch.cat((out, f))
        h = out.view(out.shape[0], -1)
        out = self.classifier(h)

        ###########################
        return out

### 4. Recurrent Neural Networks Model [pts. 20]
아래 그림과 같은 RNN 모델을 완성하시오.

(힌트, input data의 dimension의 문제가 발생할 경우 `.squeeze()` method를 사용하여 해결 할 수 있음.)

---

Complete the RNN model as shown in the figure below.

(Hint, if there is a problem with the dimension of input data, it can be solved by using the `.squeeze()` method.)

---

<div>
<img src="https://github.com/choi-hyunsoo/machine_learnig_final/blob/main/rnn.png?raw=true" align="center" width="50%"/>
</div>

In [ ]:
class RNN(nn.Module):    
    def __init__(self, output_dim, input_dim=32,
                 num_layers=3, hidden_dim=512):
        super(RNN, self).__init__()

        #### complete the code ####

        self.output_dim = output_dim
        self.input_dim = input_dim
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim

        self.rnn = nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)

        ###########################
    
    def forward(self, x):
        #### complete the code ####

        hid = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        out, hid = self.rnn(x, hid)
        out = self.fc(out)

        ###########################
        return out       

### Train the defined model and evalute [**no need to modify**]

In [ ]:
for i, arch in enumerate([MLP, CNN, CNNwithSC, RNN]):
  try:
    print(f'----------------------{i}th model----------------------------')
    model = arch(output_dim=10).apply(initialize_parameters)
    
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    
    model = model.to(device)
    print(f'The model has {count_parameters(model):,} trainable parameters')

    model = train_all(model, train_iterator, valid_iterator, optimizer, criterion, device)

    test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

    print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')
    print(f'-----------------------------------------------------------\n')

  except:
    print(f'Something wrong with {i}th model')    
    print(f'-----------------------------------------------------------\n')

----------------------0th model----------------------------
The model has 1,579,530 trainable parameters


Epochs:   0%|          | 0/2 [00:00<?, ?it/s]

Training:   0%|          | 0/211 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.383 | Train Acc: 88.72%
	 Val. Loss: 0.362 |  Val. Acc: 89.01%


...

Training:   0%|          | 0/211 [00:00<?, ?it/s]

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

Epoch: 02 | Epoch Time: 0m 35s
	Train Loss: 0.184 | Train Acc: 94.38%
	 Val. Loss: 0.234 |  Val. Acc: 93.37%


Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

Test Loss: 0.192 | Test Acc: 94.51%
-----------------------------------------------------------

